### Cascading Classification

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.0
except Exception:
    pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display


import os
from os import listdir
from os.path import isfile, join
import pickle
import random
#importing module
import sys
sys.path.insert(0, '../data')
from datahandler_cascading import create_dataset

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [2]:
with open('../data/filenames.pkl', 'rb') as infile:
    filenames = pickle.load(infile)

with open('../data/labels.pkl', 'rb') as infile2:
    labels = pickle.load(infile2)
    
print(len(filenames), len(labels))
df = pd.concat([pd.Series(filenames, name='filenames'), pd.Series(labels, name='labels')], axis=1)

24999 24999


In [3]:
with open('../data/filenames_level2.pkl', 'rb') as infile:
    filenames2 = pickle.load(infile)

with open('../data/labels_level2.pkl', 'rb') as infile2:
    labels2 = pickle.load(infile2)

print(len(filenames2), len(labels2))
df2 = pd.concat([pd.Series(filenames2, name='filenames'), pd.Series(labels2, name='labels2')], axis=1)

24998 24998


In [4]:
df = pd.merge(df, df2, on='filenames')

In [5]:
df

,filenames,labels,labels2
0,..\..\data_tate\A00001_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,..\..\data_tate\A00002_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,..\..\data_tate\A00003_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,..\..\data_tate\A00004_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,..\..\data_tate\A00005_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...,...
24993,..\..\data_tate\T13856_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24994,..\..\data_tate\T13858_8.jpg,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24995,..\..\data_tate\T13863_8.jpg,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24996,..\..\data_tate\T13864_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [27]:
class Convolution(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Convolution, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer1 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer2 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            strides = (2,2)
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm1 = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm2 = tf.keras.layers.BatchNormalization(axis=-1)

        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer1(activation)
        activation = self.nn_batchnorm1(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer2(activation)
        activation = self.nn_batchnorm2(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        return activation


class FC(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(FC, self).__init__()
        self.flat = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(filters[0], activation='relu')
        self.dense1 = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='sigmoid')
        self.dropout = tf.keras.layers.Dropout(0.2)
        
    def call(self, input_features):
        #print(input_features)
        activation = self.flat(input_features)
        activation = self.dense(activation)
        activation = self.dropout(activation)
        activation = self.dense1(activation)
        #print(activation)
        activation = self.dropout(activation)
        activation = self.out(activation)
        #print(activation)
        return activation
    
    
class Embed(tf.keras.layers.Layer):
    def __init__(self, emb_dim=28):
        super(Embed, self).__init__()
        self.embed = tf.keras.layers.Embedding(15, emb_dim*emb_dim, input_length=1)
        self.reshape = tf.keras.layers.Reshape((emb_dim,emb_dim))
        
        
    def call(self, input_features):
        #print(input_features.shape)
        activation = tf.argmax(input_features, axis=1)
        #print(activation.shape)
        activation = self.embed(activation)
        #print(activation.shape)
        activation = self.reshape(activation)
        return activation
    

class FullModel(tf.keras.Model):
    def __init__(self, emb_dim=28, filters_conv=[64,128], filters_dense=[2000,500,15], filters_dense1=[2000,500,141]):
        super(FullModel, self).__init__()
        self.convolution = Convolution(filters_conv)
        self.dense = FC(filters_dense)
        self.embed = Embed(emb_dim)
        self.dense1 = FC(filters_dense1)

    def call(self, input_features):
        activation = self.convolution(input_features)
        activation_ = self.dense(activation)
        #print(activation.shape)
        activation1 = self.embed(activation_)
        activation1 = tf.expand_dims(activation1, axis=3)
        #print(activation.shape, activation1.shape)
        activation2 = tf.concat([activation, activation1], 3)#do something that puts the conv and the embedding together
        #print(activation2)
        activation2 = self.dense1(activation2)
        #print(activation2)
        return [activation_, activation2]

class PretrainedModel(tf.keras.Model):
    def __init__(self, emb_dim=7, filters_dense=[2000,500,15], filters_dense1=[2000,500,141]):
        super(PretrainedModel, self).__init__()
        self.convolution = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
        self.convolution.trainable = False
        self.dense = FC(filters_dense)
        self.embed = Embed(emb_dim)
        self.dense1 = FC(filters_dense1)

    def call(self, input_features):
        activation = self.convolution(input_features)
        activation_ = self.dense(activation)
        #print(activation.shape)
        activation1 = self.embed(activation_)
        activation1 = tf.expand_dims(activation1, axis=3)
        #print(activation.shape, activation1.shape)
        activation2 = tf.concat([activation, activation1], 3)#do something that puts the conv and the embedding together
        #print(activation2)
        activation2 = self.dense1(activation2)
        #print(activation2)
        return [activation_, activation2]



model = FullModel()

pretrained_model = PretrainedModel()

model.build(input_shape=(32, 224, 224, 3))
pretrained_model.build(input_shape=(32, 224, 224, 3))
#final_model.build(input_shape=(32,224,224,4))
print(model.summary(), pretrained_model.summary())

Model: "full_model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_9 (Convolution)  multiple                  224512    
_________________________________________________________________
fc_20 (FC)                   multiple                  1266015   
_________________________________________________________________
embed_10 (Embed)             multiple                  11760     
_________________________________________________________________
fc_21 (FC)                   multiple                  1331141   
Total params: 2,833,428
Trainable params: 2,832,788
Non-trainable params: 640
_________________________________________________________________
Model: "pretrained_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
______________

In [28]:
def loss(model, x_train, y_train, _y_train):
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss_binary = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    intermidiate_loss = loss_binary(model(x_train)[0], y_train)
    reconstruction_error = loss_object(model(x_train)[1], _y_train)
    return tf.math.add(reconstruction_error,intermidiate_loss)


def train(loss, model, opt, x_train=None, y_train=None, _y_train=None):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, x_train, y_train, _y_train), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [29]:
train_x = list(df['filenames'][:18000])
train_y = list(df['labels'][:18000])
train_y_ = list(df['labels2'][:18000])
val_x = list(df['filenames'][18000:21000])
val_y = list(df['labels'][18000:21000])
val_y_ = list(df['labels2'][18000:21000])
test_x = list(df['filenames'][21000:])
test_y = list(df['labels'][21000:])
test_y_ = list(df['labels2'][21000:])

In [30]:
train_generator = create_dataset(train_x, train_y, train_y_)
val_generator = create_dataset(val_x, val_y, val_y_)
test_generator = create_dataset(test_x, test_y, test_y_, BATCH_SIZE=1)

In [ ]:
tf.compat.v1.enable_eager_execution()
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
opt = tf.keras.optimizers.Adam() #learning_rate=learning_rate: will add in future version
num_epochs = 20

model = PretrainedModel()

for i in range(num_epochs):
    for batch in range(len(train_x)//56):
        features, labels1, labels2 = next(iter(train_generator))
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()
        train(loss, model, opt, x_train=features, y_train=labels1, _y_train=labels2)
        loss_values = loss(model, x_train=features, y_train=labels1, _y_train=labels2)
        print(loss_values)
        train_loss_results.append(loss_values)
        print('iter end', batch)
    print('epoch end', i)

tf.Tensor(1.3408604264259338, shape=(), dtype=float64)
iter end 0


In [17]:
import pickle
predictions = []
for i in range(1):
    for batch in range(len(test_x)):
        features, labels1, labels2 = next(iter(test_generator))
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()
        loss_values = loss(model, x_train=features, y_train=labels1, _y_train=labels2)
        predictions.append((model(features)[1].numpy(), labels2.numpy()))
        print('iter end', batch)
    print('epoch end', i)
    
with open('./results/predictions_cascading.pkl', 'wb') as outfile:
    pickle.dump(predictions, outfile)

[(array([[1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9926305e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        9.9999607e-01, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9994540e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 2.071559

[(array([[1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9926305e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        9.9999607e-01, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9994540e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 2.071559

[(array([[1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9926305e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        9.9999607e-01, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.9994540e-01, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 2.071559

KeyboardInterrupt: 